# SW중심대학 공동 AI 경진대회 <본선>

1. 평가 산식

> - Accuracy (정확도)

2. 본선 평가

![](https://dacon.s3.ap-northeast-2.amazonaws.com/competition/235970/editor-image/166138683752218.jpeg)

3. 개인 또는 팀 참여 규칙

> - 본선 진출자는 예선과 동일한 팀원 및 팀명으로 참가해야 함
> - 본선 진출자 전원은 9/16(금)까지 재학증명서 또는 휴학증명서를 dacon@dacon.io 메일로 제출하야 함 [링크]
> ※ 허위 참가사실이 확인되거나 혹은 기간내 서류 제출을 하지 않을 시, 소속 팀 전체가 대회 탈락 처리됨을 유의 부탁드립니다.
> ※ 또한, 대회 기간 도중 팀원 중 졸업생 신분의 참가자가 포함되어있는 것이 확인될 경우에도 해당 팀 전체가 대회 탈락 처리됩니다.

 

4. 외부 데이터 및 사전 학습 모델 사용

> - 비상업적 용도로 사용할 수 있는 외부 데이터 사용 가능
> - 사용에 문제가 없는 사전 학습 모델(Pre-trained Model) 사용 가능


5. 유의 사항

> - 1일 최대 제출 횟수: 3회
> - 사용 가능 언어: Python, R
> - 모델 학습에서 평가 데이터셋(Test Dataset) 활용(Data Leakage) 시 실격 처리 됨 (참조 : 링크)
> - 답안을 수기로 작성하는 경우 실격 처리 됨
> - 다른 팀과의 아이디어 또는 코드 쉐어링이 의심되는 경우 데이콘에서 코드를 요청할 수 있으며, 기간 내 코드를 제출하지 않거나 치팅이 확인되는 경우 실격 처리 됨
> - 재학/휴학 증명서, 발표 자료 및 코드를 요청한 일자에 제출하지 않은 경우 실격 처리 됨
> - 최종 순위는 선택된 파일 중에서 채점되므로 참가자는 제출 창에서 자신이 최종적으로 채점을 받고 싶은 파일을 선택해야 함
> - 소프트웨어중심대학 공동 AI 경진대회 추진위원회는 대회 참가팀의 부정 행위를 금지하고 있으며, 이와 관련하여 본 경기를 위탁 운영하는 데이콘의 다른 경진대회에서 부정 제출 이력이 있는 경우에는 평가가 제한됩니다.
> - 자세한 사항은 아래의 링크를 참고해 주시기 바랍니다. https://dacon.io/notice/notice/13

6. 토론(질문)

> - 대회 운영 및 데이터 이상에 관련된 질문 외에는 답변을 드리지 않습니다.
> - 기타 질문은 토론 페이지를 통해 자유롭게 토론해주시기 바랍니다.
> - 소프트웨어중심대학 공동 AI 경진대회 추진위원회의 답변을 희망하시는 경우에는 토크 게시판의 대회 문의 게시글에 댓글을 달아주시면 검토하여 답변을 드리겠습니다.


## 모듈 및 설정

In [1]:
import os
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from jamo import h2j, j2hcj
from join_jamos import join_jamos
from trdg.generators import GeneratorFromStrings
from sklearn.model_selection import train_test_split
from deep_text_recognition_benchmark import create_lmdb_dataset

Missing modules for handwritten text generation.


In [2]:
def save_gt(filename, img_path_list, text_list):
    with open(filename, 'w', encoding='utf-8') as f:
        for img_path, text in zip(img_path_list, text_list):
            f.write(f'{img_path}\t{j2hcj(h2j(text))}\n')

In [29]:
def data_generate(char, fonts_list, count=1, orientation=0):
    lengths = np.random.randint(1, 10, count)
    strings = [''.join(list(np.random.choice(char, length))) for length in lengths]
    fonts = np.random.choice(fonts_list, count)
    size = int(np.random.normal(300, 50))
    blur = np.random.choice([True, False])
    background_type = np.random.choice([0, 1, 2])
    distorsion_type = np.random.choice([0, 1, 2, 3])
    distorsion_orientation = np.random.choice([0, 1, 2])
    text_color = f'#{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}'
    character_spacing = np.random.randint(0, 100)
    skewing_angle = np.random.randint(0, 10)
    margins = (
        size * np.random.randint(-5, 15) // 100,
        size * np.random.randint(-5, 15) // 100,
        size * np.random.randint(-5, 15) // 100,
        size * np.random.randint(-5, 15) // 100
    )
    stroke_width = np.random.choice([0, 0, 0, 3, 6])
    stroke_fill = f'#{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}{np.random.randint(0, 256):02X}'

    generator = GeneratorFromStrings(
        strings,
        fonts=fonts,
        count=count,
        language='ko',
        size=size,
        skewing_angle=skewing_angle,
        random_skew=True,
        blur=blur,
        background_type=background_type,
        distorsion_type=distorsion_type,
        distorsion_orientation=distorsion_orientation,
        text_color=text_color,
        orientation=orientation,
        character_spacing=character_spacing,
        margins=margins,
        stroke_width=stroke_width,
        stroke_fill=stroke_fill,
    )

    img_list = []
    lbl_list = []

    for img, lbl in generator:
        img_list.append(np.array(img))
        lbl_list.append(lbl)

    return img_list, lbl_list

## 데이터

### 폴더 및 파일명

In [50]:
inputs = 'inputs'
outputs = 'outputs'
processing = 'processing'

train_csv = 'train.csv'
test_csv = 'test.csv'
submission_csv = 'sample_submission.csv'

separation = 'separation'
generate_jamo = 'generate_jamo'
generate_freq = 'generate_freq'
lmdb = 'lmdb'

horizontal = 'horizontal'
vertical = 'vertical'

train_gt = 'train_gt.txt'
test_gt = 'test_gt.txt'
training_gt = 'training_gt.txt'
validation_gt = 'validation_gt.txt'
generate_gt = 'generate_gt.txt'

### CSV 파일 가져오기

In [4]:
train = pd.read_csv(os.path.join(inputs, train_csv))
test = pd.read_csv(os.path.join(inputs, test_csv))
submission = pd.read_csv(os.path.join(outputs, submission_csv))

### 새로운 텍스트 생성

In [27]:
first = [*'ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ']
middle = [*'ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ']
last = [''] + [*'ㄱㄲㄳㄴㄵㄶㄷㄹㄺㄻㄼㄽㄾㄿㅀㅁㅂㅄㅅㅆㅇㅈㅊㅋㅌㅍㅎ']

jamo_ko = [join_jamos(f'{f}{m}{l}') for f in first for m in middle for l in last]
freq_ko = [*'가각간갇갈감갑값갓강갖같갚갛개객걀걔거걱건걷걸검겁것겉게겨격겪견결겹경곁계고곡곤곧골곰곱곳공과관광괜괴굉교구국군굳굴굵굶굽궁권귀귓규균귤그극근글긁금급긋긍기긴길김깅깊까깍깎깐깔깜깝깡깥깨꺼꺾껌껍껏껑께껴꼬꼭꼴꼼꼽꽂꽃꽉꽤꾸꾼꿀꿈뀌끄끈끊끌끓끔끗끝끼낌나낙낚난날낡남납낫낭낮낯낱낳내냄냇냉냐냥너넉넌널넓넘넣네넥넷녀녁년념녕노녹논놀놈농높놓놔뇌뇨누눈눕뉘뉴늄느늑는늘늙능늦늬니닐님다닥닦단닫달닭닮담답닷당닿대댁댐댓더덕던덜덟덤덥덧덩덮데델도독돈돌돕돗동돼되된두둑둘둠둡둥뒤뒷드득든듣들듬듭듯등디딩딪따딱딴딸땀땅때땜떠떡떤떨떻떼또똑뚜뚫뚱뛰뜨뜩뜯뜰뜻띄라락란람랍랑랗래랜램랫략량러럭런럴럼럽럿렁렇레렉렌려력련렬렵령례로록론롬롭롯료루룩룹룻뤄류륙률륭르른름릇릎리릭린림립릿링마막만많말맑맘맙맛망맞맡맣매맥맨맵맺머먹먼멀멈멋멍멎메멘멩며면멸명몇모목몬몰몸몹못몽묘무묵묶문묻물뭄뭇뭐뭘뭣므미민믿밀밉밌및밑바박밖반받발밝밟밤밥방밭배백뱀뱃뱉버번벌범법벗베벤벨벼벽변별볍병볕보복볶본볼봄봇봉뵈뵙부북분불붉붐붓붕붙뷰브븐블비빌빔빗빚빛빠빡빨빵빼뺏뺨뻐뻔뻗뼈뼉뽑뿌뿐쁘쁨사삭산살삶삼삿상새색샌생샤서석섞선설섬섭섯성세섹센셈셋셔션소속손솔솜솟송솥쇄쇠쇼수숙순숟술숨숫숭숲쉬쉰쉽슈스슨슬슴습슷승시식신싣실싫심십싯싱싶싸싹싼쌀쌍쌓써썩썰썹쎄쏘쏟쑤쓰쓴쓸씀씌씨씩씬씹씻아악안앉않알앓암압앗앙앞애액앨야약얀얄얇양얕얗얘어억언얹얻얼엄업없엇엉엊엌엎에엔엘여역연열엷염엽엿영옆예옛오옥온올옮옳옷옹와완왕왜왠외왼요욕용우욱운울움웃웅워원월웨웬위윗유육율으윽은을음응의이익인일읽잃임입잇있잊잎자작잔잖잘잠잡잣장잦재쟁쟤저적전절젊점접젓정젖제젠젯져조족존졸좀좁종좋좌죄주죽준줄줌줍중쥐즈즉즌즐즘증지직진질짐집짓징짙짚짜짝짧째쨌쩌쩍쩐쩔쩜쪽쫓쭈쭉찌찍찢차착찬찮찰참찻창찾채책챔챙처척천철첩첫청체쳐초촉촌촛총촬최추축춘출춤춥춧충취츠측츰층치칙친칠침칫칭카칸칼캄캐캠커컨컬컴컵컷케켓켜코콘콜콤콩쾌쿄쿠퀴크큰클큼키킬타탁탄탈탑탓탕태택탤터턱턴털텅테텍텔템토톤톨톱통퇴투툴툼퉁튀튜트특튼튿틀틈티틱팀팅파팎판팔팝패팩팬퍼퍽페펜펴편펼평폐포폭폰표푸푹풀품풍퓨프플픔피픽필핏핑하학한할함합항해핵핸햄햇행향허헌험헤헬혀현혈협형혜호혹혼홀홈홉홍화확환활황회획횟횡효후훈훌훔훨휘휴흉흐흑흔흘흙흡흥흩희흰히힘']
fonts_list = list(map(lambda x : os.path.join('trdg/fonts/ko/', x), os.listdir('trdg/fonts/ko/')))

In [30]:
img_list, lbl_list = [], []
img_path_list, text_list = [], []
idx = 0

for _ in tqdm(range(10000)):
    img_tmp, lbl_tmp = data_generate(jamo_ko, fonts_list, orientation=1)

    for img, lbl in zip(img_tmp, lbl_tmp):
        img_path = f'./{generate_jamo}/{idx:06d}.png'
        if img.ndim != 0:
            os.makedirs(os.path.dirname(os.path.join(processing, generate_jamo, horizontal, img_path)), exist_ok = True)
            cv2.imwrite(os.path.join(processing, generate_jamo, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
            img_path_list.append(img_path)
            text_list.append(lbl)
            idx = idx + 1
    
save_gt(os.path.join(processing, generate_jamo, horizontal, generate_gt), img_path_list, text_list)

100%|██████████| 10000/10000 [5:22:55<00:00,  1.94s/it]  


In [31]:
img_list, lbl_list = [], []
img_path_list, text_list = [], []
idx = 0

for _ in tqdm(range(10000)):
    img_tmp, lbl_tmp = data_generate(jamo_ko, fonts_list, orientation=0)

    for img, lbl in zip(img_tmp, lbl_tmp):
        img_path = f'./{generate_jamo}/{idx:06d}.png'
        if img.ndim != 0:
            os.makedirs(os.path.dirname(os.path.join(processing, generate_jamo, vertical, img_path)), exist_ok = True)
            cv2.imwrite(os.path.join(processing, generate_jamo, vertical, img_path), img)
            img_path_list.append(img_path)
            text_list.append(lbl)
            idx = idx + 1
    
save_gt(os.path.join(processing, generate_jamo, vertical, generate_gt), img_path_list, text_list)

100%|██████████| 10000/10000 [3:28:10<00:00,  1.25s/it] 


In [32]:
img_list, lbl_list = [], []
img_path_list, text_list = [], []
idx = 0

for _ in tqdm(range(10000)):
    img_tmp, lbl_tmp = data_generate(freq_ko, fonts_list, orientation=1)

    for img, lbl in zip(img_tmp, lbl_tmp):
        img_path = f'./{generate_freq}/{idx:06d}.png'
        if img.ndim != 0:
            os.makedirs(os.path.dirname(os.path.join(processing, generate_freq, horizontal, img_path)), exist_ok = True)
            cv2.imwrite(os.path.join(processing, generate_freq, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
            img_path_list.append(img_path)
            text_list.append(lbl)
            idx = idx + 1
    
save_gt(os.path.join(processing, generate_freq, horizontal, generate_gt), img_path_list, text_list)

100%|██████████| 10000/10000 [5:26:03<00:00,  1.96s/it]  


In [33]:
img_list, lbl_list = [], []
img_path_list, text_list = [], []
idx = 0

for _ in tqdm(range(10000)):
    img_tmp, lbl_tmp = data_generate(freq_ko, fonts_list, orientation=0)

    for img, lbl in zip(img_tmp, lbl_tmp):
        img_path = f'./{generate_freq}/{idx:06d}.png'
        if img.ndim != 0:
            os.makedirs(os.path.dirname(os.path.join(processing, generate_freq, vertical, img_path)), exist_ok = True)
            cv2.imwrite(os.path.join(processing, generate_freq, vertical, img_path), img)
            img_path_list.append(img_path)
            text_list.append(lbl)
            idx = idx + 1
    
save_gt(os.path.join(processing, generate_freq, vertical, generate_gt), img_path_list, text_list)

100%|██████████| 10000/10000 [3:14:08<00:00,  1.16s/it] 


### 모델 튜닝 및 최종 학습 데이터셋 생성

In [7]:
vertical_fixed = set([
    'train_00349.png', 'train_00351.png', 'train_00352.png', 'train_00480.png', 'train_00489.png',
    'train_02829.png', 'train_03289.png', 'train_03544.png', 'train_03545.png', 'train_04160.png', 
    'train_04240.png', 'train_04265.png', 'train_04352.png', 'train_04842.png', 'train_05205.png', 
    'train_05505.png', 'train_06115.png', 'train_06209.png', 'train_06218.png', 'train_06841.png', 
    'train_09293.png', 'train_10097.png', 'train_12088.png'
])
horizontal_fixed = set([
    'train_01218.png', 'train_01220.png', 'train_01549.png', 'train_01551.png', 'train_01637.png', 
    'train_05147.png', 'train_05442.png', 'train_06117.png'
])
remove = set([
    'train_00885.png', 'train_00887.png', 'train_01282.png', 'train_01637.png', 'train_01679.png', 
    'train_01681.png', 'train_02282.png', 'train_02372.png', 'train_04180.png', 'train_04320.png', 
    'train_04946.png', 'train_08290.png', 'train_08334.png', 'train_08358.png', 'train_08370.png', 
    'train_08371.png', 'train_08372.png', 'train_08393.png', 'train_08394.png', 'train_08396.png', 
    'train_08412.png', 'train_08448.png', 'train_08926.png', 'train_09938.png', 'train_10315.png', 
    'train_11220.png', 'train_11265.png', 'train_11301.png', 'train_11353.png', 'train_11448.png', 
    'train_11467.png', 'train_11478.png', 'train_11645.png', 'train_11681.png', 'train_11729.png', 
    'train_11761.png', 'train_11859.png', 'train_11870.png', 'train_11979.png', 'train_11992.png', 
    'train_12057.png'
])

In [40]:
horizontal_img_path = []
horizontal_text = []
vertical_img_path = []
vertical_text = []

for img_path, text in tqdm(list(zip(train['img_path'], train['text']))):
    img_name = os.path.basename(img_path)
    img = cv2.imread(os.path.join(inputs, img_path))

    if len(text) == 1:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, horizontal, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        horizontal_img_path.append(img_path)
        horizontal_text.append(text)
        os.makedirs(os.path.dirname(os.path.join(processing, separation, vertical, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, vertical, img_path), img)
        vertical_img_path.append(img_path)
        vertical_text.append(text)
        continue

    if img_name in horizontal_fixed:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, horizontal, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        horizontal_img_path.append(img_path)
        horizontal_text.append(text)
        continue
    elif img_name in vertical_fixed:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, vertical, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, vertical, img_path), img)
        vertical_img_path.append(img_path)
        vertical_text.append(text)
        continue
    elif img_name in remove:
        continue

    height, weight, color = img.shape
    if height > weight:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, horizontal, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        horizontal_img_path.append(img_path)
        horizontal_text.append(text)
    else: 
        os.makedirs(os.path.dirname(os.path.join(processing, separation, vertical, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, vertical, img_path), img)
        vertical_img_path.append(img_path)
        vertical_text.append(text)

# Horizontal
save_gt(os.path.join(processing, separation, horizontal, train_gt), horizontal_img_path, horizontal_text)

X_train, X_valid, y_train, y_valid = train_test_split(horizontal_img_path, horizontal_text, test_size=0.2)

save_gt(os.path.join(processing, separation, horizontal, training_gt), X_train, y_train)
save_gt(os.path.join(processing, separation, horizontal, validation_gt), X_valid, y_valid)

# Vertical
save_gt(os.path.join(processing, separation, vertical, train_gt), vertical_img_path, vertical_text)

X_train, X_valid, y_train, y_valid = train_test_split(vertical_img_path, vertical_text, test_size=0.2)

save_gt(os.path.join(processing, separation, vertical, training_gt), X_train, y_train)
save_gt(os.path.join(processing, separation, vertical, validation_gt), X_valid, y_valid)


100%|██████████| 12159/12159 [12:15<00:00, 16.54it/s]


### 최종 예측 데이터셋 생성

In [10]:
horizontal_img_path = []
vertical_img_path = []

for img_path in tqdm(test['img_path']):
    img = cv2.imread(os.path.join(inputs, img_path))
    height, weight, color = img.shape
    if height > weight:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, horizontal, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, horizontal, img_path), cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE))
        horizontal_img_path.append(img_path)
    else:
        os.makedirs(os.path.dirname(os.path.join(processing, separation, vertical, img_path)), exist_ok = True)
        cv2.imwrite(os.path.join(processing, separation, vertical, img_path), img)
        vertical_img_path.append(img_path)

horizontal_text = ['예측' for _ in range(len(horizontal_img_path))]
vertical_text = ['예측' for _ in range(len(vertical_img_path))]

# Horizontal
save_gt(os.path.join(processing, separation, horizontal, test_gt), horizontal_img_path, horizontal_text)

# Vertical
save_gt(os.path.join(processing, separation, vertical, test_gt), vertical_img_path, vertical_text)

100%|██████████| 3669/3669 [04:24<00:00, 13.87it/s]


### LMDB 데이터셋 생성

In [ ]:
datasets_dict = [
    { # Horizontal dataset generated by jamo
        'tag': 'Horizontal dataset generated by jamo',
        'inputPath': os.path.join(processing, generate_jamo, horizontal),
        'gtFile': os.path.join(processing, generate_jamo, horizontal, generate_gt),
        'outputPath': os.path.join(processing, lmdb, horizontal, generate_jamo)
    },
    { # Vertical dataset generated by jamo
        'tag': 'Vertical dataset generated by jamo',
        'inputPath': os.path.join(processing, generate_jamo, vertical),
        'gtFile': os.path.join(processing, generate_jamo, vertical, generate_gt),
        'outputPath': os.path.join(processing, lmdb, vertical, generate_jamo)
    },
    { # Horizontal dataset generated by freq
        'tag': 'Horizontal dataset generated by freq',
        'inputPath': os.path.join(processing, generate_freq, horizontal),
        'gtFile': os.path.join(processing, generate_freq, horizontal, generate_gt),
        'outputPath': os.path.join(processing, lmdb, horizontal, generate_freq)
    },
    { # Vertical dataset generated by freq
        'tag': 'Vertical dataset generated by freq',
        'inputPath': os.path.join(processing, generate_freq, vertical),
        'gtFile': os.path.join(processing, generate_freq, vertical, generate_gt),
        'outputPath': os.path.join(processing, lmdb, vertical, generate_freq)
    },
    { # Horizontal training dataset
        'tag': 'Horizontal training dataset',
        'inputPath': os.path.join(processing, separation, horizontal),
        'gtFile': os.path.join(processing, separation, horizontal, training_gt),
        'outputPath': os.path.join(processing, lmdb, horizontal, 'training')
    },
    { # Vertical training dataset
        'tag': 'Vertical training dataset',
        'inputPath': os.path.join(processing, separation, vertical),
        'gtFile': os.path.join(processing, separation, vertical, training_gt),
        'outputPath': os.path.join(processing, lmdb, vertical, 'training')
    },
    { # Horizontal validation dataset
        'tag': 'Horizontal validation dataset',
        'inputPath': os.path.join(processing, separation, horizontal),
        'gtFile': os.path.join(processing, separation, horizontal, validation_gt),
        'outputPath': os.path.join(processing, lmdb, horizontal, 'validation')
    },
    { # Vertical validation dataset
        'tag': 'Vertical validation dataset',
        'inputPath': os.path.join(processing, separation, vertical),
        'gtFile': os.path.join(processing, separation, vertical, validation_gt),
        'outputPath': os.path.join(processing, lmdb, vertical, 'validation')
    },
    { # Horizontal train dataset
        'tag': 'Horizontal train dataset',
        'inputPath': os.path.join(processing, separation, horizontal),
        'gtFile': os.path.join(processing, separation, horizontal, train_gt),
        'outputPath': os.path.join(processing, lmdb, horizontal, 'train')
    },
    { # Vertical train dataset
        'tag': 'Vertical train dataset',
        'inputPath': os.path.join(processing, separation, vertical),
        'gtFile': os.path.join(processing, separation, vertical, train_gt),
        'outputPath': os.path.join(processing, lmdb, vertical, 'train')
    },
    { # Horizontal test dataset
        'tag': 'Horizontal test dataset',
        'inputPath': os.path.join(processing, separation, horizontal),
        'gtFile': os.path.join(processing, separation, horizontal, test_gt),
        'outputPath': os.path.join(processing, lmdb, horizontal, 'test')
    },
    { # Vertical test dataset
        'tag': 'Vertical test dataset',
        'inputPath': os.path.join(processing, separation, vertical),
        'gtFile': os.path.join(processing, separation, vertical, test_gt),
        'outputPath': os.path.join(processing, lmdb, vertical, 'test')
    }
]

In [56]:
for dataset_dict in datasets_dict:
    print(dataset_dict['tag'])
    create_lmdb_dataset.createDataset(
        inputPath = dataset_dict['inputPath'],
        gtFile = dataset_dict['gtFile'],
        outputPath = dataset_dict['outputPath']
    )

Horizontal dataset generated by jamo
Written 1000 / 9399
Written 2000 / 9399
Written 3000 / 9399
Written 4000 / 9399
Written 5000 / 9399
Written 6000 / 9399
Written 7000 / 9399
Written 8000 / 9399
Written 9000 / 9399
Created dataset with 9399 samples
Vertical dataset generated by jamo
Horizontal dataset generated by freq
Written 1000 / 10000
Written 2000 / 10000
Written 3000 / 10000
Written 4000 / 10000
Written 5000 / 10000
Written 6000 / 10000
Written 7000 / 10000
Written 8000 / 10000
Written 9000 / 10000
Written 10000 / 10000
Created dataset with 10000 samples
Vertical dataset generated by freq
Horizontal training dataset
Written 1000 / 1796
Created dataset with 1796 samples
Vertical training dataset
Horizontal validation dataset
Created dataset with 450 samples
Vertical validation dataset
Horizontal train dataset
Written 1000 / 2246
Written 2000 / 2246
Created dataset with 2246 samples
Vertical train dataset
Horizontal test dataset
Created dataset with 445 samples
Vertical test data

## 모델

### deep_text_recognition_benchmark

아래의 명령어를 실행해 수평 / 수직 방향에 대한 간판이미지를 따로 학습   
(메모리 누수 문제로 인해서 Jupyter notebook 안에서 실행하지 않음)

python deep_text_recognition_benchmark/train.py